# Bathymetry For MOM6 Example

In this example, I test a concept that involves replicating the NEP7 MOM6 bathymetry grid file. We will take the bathymetry file, match it with an existing grid file spawned from the grid generation tool, and output a new bathymetric file. Note that the bathy topography is half the resolution of the hgrid. Per Kate Hedstrom - The ocean_hgrid file is the supergrid while the model splits that up into the u-point, v-point, h-point and q-point grids. You only need h on the h-points.

### Load NEP7 Reference Bathy File

In [15]:
import xarray as xr

nep7 = xr.open_dataset("/Users/james/Documents/Github/esm_lab/gridTools/nep7_grid/ocean_topog.nc")


In [16]:
#nep7.depth.plot()

### Load the MOM6 GridGen Gridfile

In [18]:
grid = xr.open_dataset("/Users/james/Downloads/gridFile.nc")
#grid = grid.rio.write_crs(grid.attrs['proj'])
#grid = grid.rio.set_spatial_dims(y_dim='ny', x_dim='nx', inplace=True)

### Load the Bathymetry File & Slice it down to the size we need

In [24]:
import numpy as np

gebco = xr.open_dataset("/Users/james/Downloads/gebco_2020_netcdf/GEBCO_2020.nc").sel(lat=slice(np.min(grid.y.values), np.max(grid.y.values)), lon=slice(np.min(grid.x.values), np.max(grid.x.values)))



# get 2D versions of the lat and lon variables (note the -180 here!)
lon2d, lat2d = np.meshgrid(gebco.lon.values, gebco.lat.values)

#da.assign_coords({"lon_2": ("lon", lon_2)})
gebco = gebco.assign_coords({"lon2d" : (("lat", "lon"), lon2d)})
gebco = gebco.assign_coords({"lat2d" : (("lat", "lon"), lat2d)})

In [25]:
#gebco = gebco.rio.write_crs("EPSG:4326")
#gebco = gebco.rio.set_spatial_dims(y_dim='lat', x_dim='lon', inplace=True)

### Downsample the Grid File

For reasons specified above, we need to downsample the grid file to half in order to create an appropriate file for the bathymetry

In [26]:
coGrid = grid.coarsen(nx=2, ny=2, nxp=2, nyp=2, boundary='pad').mean()


### Resample gebco file to coarse grid dimensions

In [27]:
# set spatial dimensions for rioxarray
# THIS IS WHERE I'M STUCK - we have an irregular grid, and ny/nx are just the number of center points - the latitude changes with longitude for each center point
#coGrid = coGrid.rio.set_spatial_dims(y_dim='ny', x_dim='nx', inplace=True)

# write the rio crs for rioxarray
#coGrid = coGrid.rio.write_crs(grid.attrs['proj'])


#gebco = gebco.rio.write_crs("EPSG:4326")
#gebco = gebco.rio.set_spatial_dims(y_dim='lat', x_dim='lon', inplace=True)

# reproject match - Reproject a DataArray object to match the resolution, projection, and region of another DataArray.
#gebco_match = gebco.rio.reproject_match(coGrid)
#gebco_match

gebco = gebco.rename({"lon" : "x", "lat" : "y"})
gebco, coGrid

(<xarray.Dataset>
 Dimensions:    (x: 10036, y: 8610)
 Coordinates:
   * x          (x) float64 -153.3 -153.3 -153.3 -153.3 ... -111.5 -111.5 -111.5
   * y          (y) float64 22.06 22.06 22.06 22.07 ... 57.91 57.92 57.92 57.93
     lon2d      (y, x) float64 -153.3 -153.3 -153.3 ... -111.5 -111.5 -111.5
     lat2d      (y, x) float64 22.06 22.06 22.06 22.06 ... 57.93 57.93 57.93
 Data variables:
     elevation  (y, x) int16 ...
 Attributes:
     Conventions:  CF-1.6
     title:        The GEBCO_2020 Grid - a continuous terrain model for oceans...
     institution:  On behalf of the General Bathymetric Chart of the Oceans (G...
     source:       The GEBCO_2020 Grid is the latest global bathymetric produc...
     history:      Information on the development of the data set and the sour...
     references:   DOI: 10.5285/a29c5465-b138-234d-e053-6c86abc040b9
     comment:      The data in the GEBCO_2020 Grid should not be used for navi...
     node_offset:  1.0,
 <xarray.Dataset>
 Dimens

In [28]:
# try xesmf

import xesmf as xe

regridder = xe.Regridder(gebco, coGrid, 'bilinear')
dr_out = regridder(gebco)

KeyError: 'lon'

In [2]:
gebco

NameError: name 'gebco' is not defined